In [37]:
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt

# активационная функция
def sigmoid(z):
    return 1.0 / (1.0 + np.exp(-z))

# производная активационной функции
def sigmoid_prime(z):
    return sigmoid(z) * (1 - sigmoid(z))

# квадратичная целевая функция
# test_data - кортеж из входных значений и списка значений классов
def cost_function(network, test_data):
    c = 0
    for example, y in test_data:
        c += np.sum((network.feedforward(example) - y) ** 2)
    return c /(2 * len(test_data))


In [38]:
class Network:
    def __init__(self, shape, activation_function, activation_function_derivative, debug= True):
        self.shape = shape
        self.biases = [np.random.randn(y, 1) for y in shape[1:]]
        self.weights = [np.random.randn(y, x) for x, y in zip(shape[:-1], shape[1:])]
        self.activation_function = activation_function
        self.activation_function_derivative = activation_function_derivative
        self.debug = debug

    # прогнать до конца примеры из input_matrix
    def feedforward(self, input_matrix):
        for b, w in zip(self.biases, self.weights):
            # weigts - (массив матриц)
            input_matrix = self.activation_function(np.dot(w, input_matrix) + b)
        return input_matrix

    # обновление параметров нейронной сети (веса, смещения), сделав шаг градиентного спуска
    # на основе алгоритма обратного распространения ошибки
    # alpha - learning rate
    def back_prop_step(self, data, alpha):
        # значения dJ/db для каждого слоя
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        # значения dJ/dw (ошибки) для каждого слоя
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # для каждого примера из батча применяем бек пропогейшн
        for x, y in data:
            delta_nabla_b, delta_nabla_w = self.back_prop_single_example(x, y)
            nabla_b = [nb + dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]

        eps = alpha / len(data)

        # обновляем параметры сети
        self.weights = [w - eps * nw for w, nw in zip(self.weights, nabla_w)]
        self.biases  = [b - eps * nb for b, nb in zip(self.biases,  nabla_b)]

    def prepare_data(data, classes_count):
        return [(row[:-classes_count].reshape(-1,1), row[-classes_count:].reshape(-1,1)) for row in data]

    # алгоритм градиентного спуска
    def SGD(self, data_in, epochs, alpha, classes_count):
        prepared_data = Network.prepare_data(data_in, classes_count)
        errors = []
        
        for epoch in range(epochs):
            self.back_prop_step(prepared_data, alpha)
            error = cost_function(self, prepared_data)
            if self.debug:
                print(f'epoch: {epoch} -  error:{error}')
            errors.append(error)
            

        return errors
    
    # возвращает вектор частных производных квадратичной целевой функции по активациям выходного слоя
    def cost_derivative(self, output_activations, y):
        return output_activations - y

    # алгоритм обратного распространения ошибки для одного примера из тренировочной выборки
    # возвращает кортеж (nabla_b, nabla_w) - градиентов по слоям по смещениям и весам соответственно
    def back_prop_single_example(self, x, y):
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]

        # прямое распространение (forward pass)
        # массив векторов активаций нейронов
        activations = [x]
        # массив векторов сумматорных функций от активаций предыдущих слоев
        summatories = []
        # b - вектор смещений нейронов данного слоя
        # w - матрица весов, входящих в данный слой
        for b, w in zip(self.biases, self.weights):
            summatories.append(np.dot(w, activations[-1]) + b)
            activation = self.activation_function(summatories[-1])
            activations.append(activation)

        # обратное распространение (backward pass)
        
        # ошибка для выходного слоя
        delta = self.cost_derivative(activations[-1], y) * self.activation_function_derivative(summatories[-1])
        # производная J по биасам выходного слоя
        nabla_b[-1] = delta
        # производная J по весам выходного слоя
        nabla_w[-1] = delta.dot(activations[-2].T)

        # Здесь l = 1 означает последний слой, l = 2 - предпоследний и так далее.  
        for l in range(2, len(self.shape)):
            derivative = self.activation_function_derivative(summatories[-l])
            # ошибка на слое L-l
            delta = derivative * self.weights[-l + 1].T.dot(delta)
            # производная J по смещениям L-l-го слоя
            nabla_b[-l] = delta
            # производная J по весам L-l-го слоя
            nabla_w[-l] = delta.dot(activations[-l - 1].T) 
        return nabla_b, nabla_w

In [45]:
def draw_class(index, normalized_data, class_count):
    class_n = [float(network.feedforward(row[:-class_count].reshape(-1,1))[index]) for row in normalized_data]
    class_real = [float(row[-class_count:].reshape(-1,1)[index]) for row in normalized_data]

    plt.scatter(range(len(class_real)), class_real)
    plt.plot(class_n,'g')
    return class_n

def rmse(network, train_data):
    print(cost_function(network, train_data))
    return np.sqrt(cost_function(network, train_data))

def get_normalized_data(data, classes_count):
    normalized = (data / data.max())
    normalized.iloc[:, :-classes_count] = normalized.iloc[:, :-classes_count].fillna(0)
    mean = normalized.iloc[:, -classes_count:].mean()
    normalized.iloc[:, -classes_count:] = normalized.iloc[:, -classes_count:].fillna(mean)
    return normalized       

In [46]:
data = pd.read_csv('https://raw.githubusercontent.com/SmirAlex/back-propogation/master/cleared_data.csv', delimiter=';',encoding='utf-8')
train_data = data.sample(frac=0.7)
test_data = data.loc[~data.index.isin(train_data.index)]
normalized_train_data = get_normalized_data(train_data, 2)
normalized_test_data = get_normalized_data(test_data, 2)

In [48]:
pd.read_csv('https://raw.githubusercontent.com/SmirAlex/back-propogation/master/cleared_data.csv', delimiter=';')


,Рзаб,Pлин,Руст.1,Рзаб.1,Рлин,Туст,Тна шлейфе,Тзаб,Tлин,Дебит ст. конд.,Дебит воды,Дебит смеси,Дебит гааз,Дебит кон нестабильный,Дебит воды.1,Pсб.1,G_total,КГФ
0,370.1,101.800000,249.0,359.6,101.800,53.0,31.7,103.20,32.500000,83.6,0.400000,231.248840,1610.368866,131.3,0.4,92.376018,2.782623,311.9094
1,364.6,101.300000,231.0,338.1,102.400,58.8,37.6,103.00,38.600000,104.7,1.400000,309.001660,2310.226429,158.5,1.4,91.685171,3.697781,288.6003
2,357.1,101.600000,211.0,314.8,100.600,63.6,42.8,102.60,43.400000,114.3,1.900000,388.114980,3039.489680,172.3,1.9,90.204787,4.515073,248.7906
3,347.4,98.400000,187.0,291.5,99.000,64.7,46.2,102.00,46.100000,121.3,3.300000,455.214520,3824.082686,181.5,3.3,88.033555,5.217673,223.5591
4,337.7,99.200000,169.0,270.3,99.500,64.0,49.4,104.40,49.900000,129.8,4.600000,504.587780,4299.104676,190.6,4.7,88.527017,5.765092,215.1486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,307.8,98.600000,215.0,308.1,97.200,65.0,45.8,104.78,46.300000,93.5,4.700000,495.629850,4165.000000,156.8,4.8,93.165556,NaN,153.0000
89,293.6,98.500000,197.0,293.9,98.400,66.9,49.8,104.63,50.500000,106.6,6.400000,587.360940,5113.000000,174.3,6.5,92.770787,NaN,147.0000
90,309.1,92.600000,230.0,307.1,92.700,51.0,NaN,104.40,28.100000,50.5,0.448917,224.858099,1759.000000,84.7,0.4,88.724402,NaN,182.0000
91,293.9,93.100000,215.0,293.9,93.300,60.0,NaN,104.60,41.300000,72.0,1.500000,403.746900,3454.000000,123.0,1.6,89.612633,NaN,145.0000


In [50]:
normalized_test_data

,Рзаб,Pлин,Руст.1,Рзаб.1,Рлин,Туст,Тна шлейфе,Тзаб,Tлин,Дебит ст. конд.,Дебит воды,Дебит смеси,Дебит гааз,Дебит кон нестабильный,Дебит воды.1,Pсб.1,G_total,КГФ
2,0.860897,0.912177,0.708054,0.759103,0.877071,0.859459,0.868154,0.962477,0.816938,0.436236,0.316048,0.508244,0.513922,0.414183,0.311475,0.880539,0.484057,0.860321
12,0.752821,0.926426,0.711409,0.753075,0.897995,0.824324,0.835700,0.986867,0.773904,0.326326,0.431089,0.536926,0.521702,0.335817,0.442623,0.902697,0.602248,0.594778
13,0.703327,0.926811,0.651007,0.703400,0.905993,0.837838,0.886410,0.980300,0.830927,0.329045,0.525662,0.588819,0.597602,0.331010,0.524590,0.863198,0.602248,0.542908
14,0.634113,0.964049,0.557047,0.634193,0.937547,0.851351,0.997972,0.969981,0.936539,0.336134,0.583959,0.666607,0.688433,0.341827,0.590164,0.869942,0.602248,0.487580
19,0.652122,0.717825,0.543624,0.652038,0.707470,0.702703,0.689655,0.986116,0.636718,0.546329,0.070981,0.667507,0.867220,0.531731,0.065574,0.952794,0.619166,0.809038
21,0.605834,0.658156,0.506711,0.687967,0.632775,0.675676,0.000000,0.940056,0.602240,0.438444,0.056724,0.624217,0.880831,0.414663,0.049180,0.934489,0.562946,0.690315
22,0.715526,0.687473,0.644295,0.715698,0.669743,0.675676,0.000000,0.960413,0.466859,0.417055,0.054747,0.486536,0.626532,0.394231,0.049180,0.944123,0.455483,0.849576
28,1.000000,0.975322,0.909396,1.000000,0.947167,1.000000,0.000000,0.988274,1.000000,1.000000,1.000000,1.000000,1.000000,0.964183,1.000000,0.927746,1.000000,1.000000
30,0.972517,1.000000,1.000000,0.971546,0.973085,0.864865,0.649087,0.979550,0.798873,0.451414,0.244410,0.508335,0.460680,0.443029,0.245902,0.855491,0.491837,0.881236
33,0.823288,0.951681,0.843242,0.831927,1.000000,0.337838,0.000000,0.984053,0.498821,0.152664,0.000000,0.000000,0.299038,0.236058,0.000000,0.913295,0.602248,0.933663


In [51]:
network = Network([16, 60, 70, 2], sigmoid, sigmoid_prime, True)
errors = network.SGD(normalized_train_data, epochs=1000, alpha=0.01, classes_count=2)

AttributeError: 'str' object has no attribute 'reshape'

In [ ]:
fig = plt.gcf()
fig.set_size_inches(20, 20)

plt.subplot(3, 1, 1)
plt.plot(errors)

for class_index in range(2):
    plt.subplot(3, 1, 2 + class_index)
    draw_class(class_index, normalized_test_data, 2)

plt.show()
print(rmse(network, Network.prepare_data(normalized_test_data, 2)))